In [ ]:
import pandas as pd

train = pd.read_csv("data_titanic/train.csv")
train.Pclass = train.Pclass.astype(float) # to avoid DataConversionWarning

In [ ]:
train.head()

## Brief Exploration

In [ ]:
# Categorical features
train.describe(include = object)

In [ ]:
# Numerical features
train.describe()

Let's work only with the following features for simplicity:   

**Categorical**   
- Sex
- Embarked

**Numerical**  
- Survived: *our target feature* (0 = No, 1 = Yes)
- Pclass: Ticket class (1 = 1st, 2 = 2nd, 3 = 3rd)
- Age: Age in years
 
More detailed info: https://www.kaggle.com/c/titanic

In [ ]:
# Let's keep only the desired columns
train = train[['Sex','Embarked','Pclass', 'Age','Survived']]

In [ ]:
train.shape

In [ ]:
# Check for missing values
train.isna().sum()

For simplicity, we drop any row containing missing values. 

**Note**   
If you later want to experiment with composite transformers, comment out this cell and include also missing value imputation.

In [ ]:
train = train.dropna(axis=0)
train.head()

## Feature Engineering
With our current knowledge, we can try to individually implement various transformers from Scikit Learn. Let's not forget to create a holdout set!

In [ ]:
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train[['Pclass', 'Age', 'Sex', 'Embarked']],
                                                    train['Survived'], 
                                                    test_size=0.2, 
                                                    random_state=42)

### Numerical Features
The only numerical features we have are 'Pclass' and 'Age'.  
Let's scale these two features using `MinMaxScaler()`.

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train[['Pclass', 'Age']])
X_train_transformed_numerical = scaler.transform(X_train[['Pclass', 'Age']])
X_test_transformed_numerical = scaler.transform(X_test[['Pclass', 'Age']])

print(X_train_transformed_numerical.shape)
print(X_test_transformed_numerical.shape)

### Categorical Features
The categorical features we have are 'Sex' and 'Embarked'.   
We can simply one-hot encode these using `OneHotEncoder()`.

In [ ]:
encoder = preprocessing.OneHotEncoder(sparse=False)
encoder.fit(X_train[['Sex', 'Embarked']])
X_train_transformed_categorical = encoder.transform(X_train[['Sex', 'Embarked']])
X_test_transformed_categorical = encoder.transform(X_test[['Sex', 'Embarked']])

print(X_train_transformed_categorical.shape)
print(X_test_transformed_categorical.shape)

## HANDS-ON 1: Baseline Model & Model Evaluation
It's time for our first exercise! 
Before, let's concatenate the transformed numerical and categorical features into a single dataframe.

In [ ]:
X_train_transformed = np.concatenate((X_train_transformed_numerical,X_train_transformed_categorical), axis = 1)
X_test_transformed = np.concatenate((X_test_transformed_numerical,X_test_transformed_categorical), axis = 1)

print(X_train_transformed.shape)
print(X_test_transformed.shape)

In [ ]:
# TASK 1: Fit sklearn.DummyClassifier to the transformed training set.  
# Then, let the model predict for train (X_train_transformed) and holdout set (X_test_transformed).
# Store the prediction as y_pred_TRAIN_DUMMY (training set) and as y_pred_HOLDOUT_DUMMY (holdout set).

from sklearn.dummy import DummyClassifier


In [ ]:
# OPTIONAL TASK 1: Think about a simple heuristic that can be used as a baseline. 
# One possibility is to use gender and for example predict that every men or every woman has survived.
# You can store the result as y_pred_TRAIN_HEURISTIC and as y_pred_HOLDOUT_HEURISTIC.


Great! We have our first prediction! It is time to evaluate how good our model is using the [*sklearn.metrics* module.](   
https://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics)

In [ ]:
from sklearn import metrics

#TASK 2A: Display ACCURACY on TRAIN set.

#TASK 2B: Display ACCURACY on HOLDOUT set.

#OPTIONAL TASK 2C: Can you think of a better measure than accuracy based on the domain problem? If yes, use it the same way.

Great! Now we would also like to see the confusion matrix as it is always a good idea to visually confirm the quality of our predictions.

In [ ]:
#TASK 3: Display a CONFUSION MATRIX on HOLDOUT set. Hint: do not use plot_confusion_matrix but confusion_matrix only.


## HANDS-ON 2: Composite Estimators
Let's nicely wrap our feature engineering and model fitting into a nice composite estimator. We will be very simplistic and only use two steps. 
They will not nest into each other at once.

### Feature Engineering wrapped into ColumnTransformer
The two feature transformations can be easily wrapped up into a single `ColumnTransformer()` object. This will ensure that our Feature Engineering is a bit **more robust and nicely encapsulated**. Section 6.1.4 [here](https://scikit-learn.org/stable/modules/compose.html#columntransformer-for-heterogeneous-data) showcases the exact application that we intend to create.



In [ ]:
# TASK 3: Wrap MinMaxScaler and OneHotEncoder into a single ColumnTransformer. 
# The transformers should be applied to the respective numerical or categorical columns only.
# Store the resulting composite as feature_engineering
# Hint: Use the argument remainder='passthrough'

from sklearn.compose import ColumnTransformer


### Predictive Model Wrapped into Pipeline
Let's now wrap the feature engineering and the model into a single Pipeline Composite estimator. Here is some pseudocode for this:
``` 
entire_pipeline = feature_engineering -> model  
``` 

Both components are already available. From the step above we can directly reuse the object `feature_engineering`. As model, we just call a new `DummyClassifier`, just as we did before.

In [ ]:
# TASK 4: Wrap the feature engineering and the predictive model (dummy) into a single Pipeline composite estimator. 
# Store the result as entire_pipeline.
from sklearn.pipeline import Pipeline


In [ ]:
# TASK: Uncomment the line and try to train the pipeline.
# Notice that we are using untransformed data again (X_train) as the pipeline contains all necessary transformers.

# entire_pipeline.fit(X = X_train, y = y_train)

In [ ]:
# Predict for training data
y_pred_TRAIN_DUMMY = entire_pipeline.predict(X_train)

# Predict for holdout data
y_pred_HOLDOUT_DUMMY = entire_pipeline.predict(X_test)

# Results should be the same as before
print(metrics.accuracy_score(y_train, y_pred_TRAIN_DUMMY))

# Display accuracy on holdout set.
print(metrics.accuracy_score(y_test, y_pred_HOLDOUT_DUMMY))

**OPTIONAL TASK**   
The notebook 'nice_pipeline' was made to exemplify some examples of more complex pipelines. Feel free to scroll through it and learn what the process of preparing a complex composite looks like. You can then come back here and try to implement various components. For example, if I would not drop rows with missing values at the beginning of this notebook, constructing a composite would get a bit trickier. 

## HANDS-ON 3: Tree-based Models & Hyperparameter Tuning
Hold your constructed pipeline firmly! The only thing that we need to do now is to replace `DummyClassifier` with a proper learning model.   
We can start with a decision tree.

### Fitting a Learning Model – Decision Tree

In [ ]:
# TASK 5: Reuse your composite and instead of a dummy, fit a decision tree with default parameters.
# Store the result as dt_pipeline.
from sklearn.tree import DecisionTreeClassifier

# Train the pipeline
# dt_pipeline.fit(X = X_train, y = y_train)

In [ ]:
# TASK 5B: Let the pipeline predict for the training set. 
# Store the result as y_pred_TRAIN_DT.
# Also, display accuracy.


In [ ]:
# TASK 5C: Let the pipeline predict for the holdout set. 
# Store the result as y_pred_HOLDOUT_DT.
# Also, display accuracy.


Looking at the accuracy on training and holdout set, what can you infer about over model? Will it generalize well?

In [ ]:
# OPTIONAL TASK 6: Do the same steps with RandomForest with default parameters. 
# Does the RandomForest display similar results as decision tree? If not, why?
from sklearn.ensemble import RandomForestClassifier


### Tuning Hyperparameters of our Decision Tree
Time to improve the performance of our learning model by finding its optimal set of hyperparameters.  
We start by examining **which hyperparameters are available** in our decision tree pipeline.

In [ ]:
dt_pipeline.get_params()

We would like to tune `max_depth` and `min_samples_split`.  
Notice that to access them, we also need to navigate within the composite and call them as **`decision_tree`**`__max_depth`.  

In [ ]:
# TASK 7: Define a grid through which we should search. 
# Tune parameters: max_depth and min_samples_split.
# The values which you pick as parameters are up to you. You can think about them intuitively.


In [ ]:
from sklearn import tree
from sklearn.model_selection import GridSearchCV

# Model
dt_pipeline

# Searching strategy, providing grid
tuning = GridSearchCV(dt_pipeline, param_grid)

# Train
tuning.fit(X_train, y_train)

In [ ]:
# Let's get the best parameters
tuning.best_params_

In [ ]:
# TASK 8: Use the best setting of the two hyperparameters and fit a optimized decision tree. 
# Hint: Reuse the pipeline and when declaring it, specify the params.
# Store it as dt_pipeline_tuned.

... 

# Train
dt_pipeline_tuned.fit(X_train, y_train)

In [ ]:
# TASK 8B: Display accuracy on the training set of the optimized decision tree.


In [ ]:
# TASK 8C: Display accuracy on the holdout set of the optimized decision tree.


Does the optimized decision tree perform better then the one with default parameters?

### Optional Advanced TASK: Tuning Random Forest
When you are tuning a more complex model, it is good practice to search available literature on which hyperparameters should be tuned. Below I have predefined some. You can play around with the grid, for example expand or narrow it. Keep in mind that as our feature set is extremely limited, its hard for hyperparameter tuning to arrive at something meaningful.

In [ ]:
# OPTIONAL TASK 9
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Define a pipeline
rf_pipeline = Pipeline([('feature_engineering', feature_engineering), ('random_forest', RandomForestClassifier())])

# Create the parameter grid based on the results of random search 
param_grid_rf = {
    'random_forest__bootstrap': [True, False],
    'random_forest__max_depth': [3, 5, 10, 15],
    'random_forest__max_features': [2, 3],
    'random_forest__min_samples_leaf': [3, 4, 5],
    'random_forest__min_samples_split': [5, 8, 10, 12],
    'random_forest__n_estimators': [5, 10, 15, 20, 25]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf_pipeline, 
                           param_grid = param_grid_rf, 
                           cv = 3, 
                           n_jobs = -1, 
                           verbose = 2)

# Searching strategy, providing grid
tuning_rf = GridSearchCV(rf_pipeline, param_grid_rf)

# Train
tuning_rf.fit(X_train, y_train)

# Cross-validated score (more robust than holdout set most likely)
print(tuning_rf.best_score_)
print(tuning_rf.best_params_)

### Optional Advanced TASK: Check Kaggle competitions and join one of them!  
https://www.kaggle.com/